In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

from league_team_names import epl_team_dict, bundesliga_team_dict, laliga_team_dict, seriea_teams_dict

country_group_dict = pd.read_csv('resources/worldcup_country_groups.csv', ).to_dict

In [2]:
players_df = pd.read_csv('resources/fifa-18-demo-player-dataset/CompleteDataset.csv')
players_df.drop(['Unnamed: 0', 'Flag', 'Club Logo', 'Value', 'Wage', 'Photo', 'Potential', "Special"], axis=1, inplace=True)
players_df[players_df['Nationality']=='Serbia'][['Name', 'Preferred Positions']]

/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Preferred Positions
155,N. Matić,CM CDM
264,L. Fejsa,CM CDM
270,B. Ivanović,RB CB
393,S. Milinković-Savić,CAM CM
415,M. Nastasić,CB
428,D. Tadić,RM LM CAM
491,D. Basta,RM RB
530,N. Maksimović,CB
610,A. Ljajić,CAM LW
627,A. Kolarov,CB LB


In [3]:
# positions_list = list(players_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).drop_duplicates())
# positions_list

In [4]:
positions_dict = {'ST': 'Attack',
                  'RW': 'Attack',
                  'LW': 'Attack',
                  'GK': 'Defense',
                  'CDM': 'Midfield',
                  'CB': 'Defense',
                  'RM': 'Midfield',
                  'CM': 'Midfield',
                  'LM': 'Midfield',
                  'LB': 'Defense',
                  'CAM': 'Attack',
                  'RB': 'Defense',
                  'CF': 'Attack',
                  'RWB': 'Defense',
                  'LWB': 'Defense'}

In [5]:
players_df

,Name,Age,Nationality,Overall,Club,Acceleration,Aggression,Agility,Balance,Ball control,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,Cristiano Ronaldo,32,Portugal,94,Real Madrid CF,89,63,89,63,93,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,L. Messi,30,Argentina,93,FC Barcelona,92,48,90,95,95,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,Neymar,25,Brazil,92,Paris Saint-Germain,94,56,96,82,95,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,L. Suárez,30,Uruguay,92,FC Barcelona,88,78,86,60,91,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,M. Neuer,31,Germany,92,FC Bayern Munich,58,29,52,35,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R. Lewandowski,28,Poland,91,FC Bayern Munich,79,80,78,80,89,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0
6,De Gea,26,Spain,90,Manchester United,57,38,60,43,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,E. Hazard,26,Belgium,90,Chelsea,93,54,93,91,92,...,59.0,47.0,81.0,61.0,87.0,87.0,82.0,88.0,64.0,82.0
8,T. Kroos,27,Germany,90,Real Madrid CF,60,60,71,69,89,...,76.0,72.0,87.0,82.0,81.0,81.0,77.0,80.0,78.0,77.0
9,G. Higuaín,29,Argentina,90,Juventus,78,50,75,69,85,...,51.0,46.0,71.0,52.0,84.0,79.0,87.0,82.0,55.0,87.0


In [6]:
overall_score_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
overall_score_df['Preferred Positions'] = overall_score_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

team = 'Brazil'
overall_score_df[overall_score_df['Nationality'] == team].head(14).groupby(['Nationality', 'Preferred Positions']).mean()


Overall
Nationality Preferred Positions           
Brazil      Attack               87.333333
            Defense              85.250000
            Midfield             84.666667

In [7]:
epl2018 = pd.read_csv('resources/leagues/epl2018.csv')
bundesliga2018 = pd.read_csv('resources/leagues/bundesliga2018.csv')
seriea2018 = pd.read_csv('resources/leagues/seriea2018.csv')
laliga2018 = pd.read_csv('resources/leagues/laliga2018.csv')
epl2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [8]:
results_dict = {'H': 2, "D": 1, "A": 0}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
test_df = epl2018[cols].rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Results'}).copy()
test_df['Results'] = test_df['Results'].replace(results_dict)
test_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Results
0,Arsenal,Leicester,4,3,2
1,Brighton,Man City,0,2,0
2,Chelsea,Burnley,2,3,0
3,Crystal Palace,Huddersfield,0,3,0
4,Everton,Stoke,1,0,2
5,Southampton,Swansea,0,0,1
6,Watford,Liverpool,3,3,1
7,West Brom,Bournemouth,1,0,2
8,Man United,West Ham,4,0,2
9,Newcastle,Tottenham,0,2,0


In [9]:
player_overall_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
player_overall_df['Preferred Positions'] = player_overall_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

In [10]:
def get_team_stats(team, type='Club', no_of_players=13, return_df=False):
    mask = overall_score_df[type] == team
    groupby_cols = [type, 'Preferred Positions']
    team_stats_df = player_overall_df[mask].head(no_of_players).groupby(groupby_cols).mean().round(0)
    if return_df == True:
        return team_stats_df
    else:
        df = team_stats_df.reset_index().pivot(type, 'Preferred Positions')
        attack = df['Overall'].values[0][0]
        mid = df['Overall'].values[0][1]
        defense = df['Overall'].values[0][2]
        return [attack, mid, defense]

In [11]:
get_team_stats('Brazil', type='Nationality', return_df=False)

[87.0, 86.0, 85.0]

In [12]:
get_team_stats('Chelsea')

[86.0, 85.0, 84.0]

In [21]:
team_cols = ['HomeTeam', 'AwayTeam']
epl2018[team_cols] = epl2018[team_cols].replace(epl_team_dict)
bundesliga2018[team_cols] = bundesliga2018[team_cols].replace(bundesliga_team_dict)
seriea2018[team_cols] = seriea2018[team_cols].replace(seriea_teams_dict)
laliga2018[team_cols] = laliga2018[team_cols].replace(laliga_team_dict)

In [22]:
seriea2018['HomeTeam'].drop_duplicates().sort_values()

2             Atalanta
10    Benevento Calcio
3              Bologna
24            Cagliari
13       Chievo Verona
4              Crotone
18      Ferrara (SPAL)
15          Fiorentina
11               Genoa
1        Hellas Verona
5                Inter
0             Juventus
6                Lazio
16               Milan
17              Napoli
12                Roma
7            Sampdoria
8             Sassuolo
19              Torino
9              Udinese
Name: HomeTeam, dtype: object

In [23]:
results_dict = {'H': 'HomeWin', "D": 'Draw', "A": 'AwayWin'}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

league_results_df = pd.concat([epl2018, bundesliga2018, seriea2018, laliga2018])[cols]
league_results_df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Result'}, inplace=True)
league_results_df['Result'] = league_results_df['Result'].replace(results_dict)
league_results_df['Goal diff'] = league_results_df['HomeGoals'] - league_results_df['AwayGoals']

league_results_df.reset_index(drop=True, inplace=True)
league_results_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
0,Arsenal,Leicester City,4,3,HomeWin,1
1,Brighton & Hove Albion,Manchester City,0,2,AwayWin,-2
2,Chelsea,Burnley,2,3,AwayWin,-1
3,Crystal Palace,Huddersfield Town,0,3,AwayWin,-3
4,Everton,Stoke City,1,0,HomeWin,1
5,Southampton,Swansea City,0,0,Draw,0
6,Watford,Liverpool,3,3,Draw,0
7,West Bromwich Albion,Bournemouth,1,0,HomeWin,1
8,Manchester United,West Ham United,4,0,HomeWin,4
9,Newcastle United,Tottenham Hotspur,0,2,AwayWin,-2


In [24]:
# Chart this

In [25]:
league_results_df['Result'].value_counts()

HomeWin    655
AwayWin    440
Draw       351
Name: Result, dtype: int64

In [26]:
homeTeam_attack = []
homeTeam_mid = []
homeTeam_def = []
awayTeam_attack = []
awayTeam_mid = []
awayTeam_def = []
data = []


for index, row in league_results_df.iterrows():
    try:
        home_team_stats = get_team_stats(row['HomeTeam'])
        home_attack = home_team_stats[0]
        home_midfield = home_team_stats[1]
        home_defense = home_team_stats[2]

        away_team_stats = get_team_stats(row['AwayTeam'])
        away_attack = away_team_stats[0]
        away_midfield = away_team_stats[1]
        away_defense = away_team_stats[2]
                
        homeTeam_attack.append(home_attack)
        homeTeam_mid.append(home_midfield)
        homeTeam_def.append(home_defense)
        awayTeam_attack.append(away_attack)
        awayTeam_mid.append(away_midfield)
        awayTeam_def.append(away_defense)

        
    except IndexError:
        homeTeam_attack.append(None)
        homeTeam_mid.append(None)
        homeTeam_def.append(None)
        awayTeam_attack.append(None)
        awayTeam_mid.append(None)
        awayTeam_def.append(None)
        continue

In [27]:
data = [homeTeam_attack, 
        homeTeam_mid, 
        homeTeam_def, 
        awayTeam_attack,
        awayTeam_mid, 
        awayTeam_def]

feature_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

home_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

away_cols = ['awayTeam_attack', 
             'awayTeam_mid',
             'awayTeam_def',
             'homeTeam_attack', 
             'homeTeam_mid', 
             'homeTeam_def']

rename_cols = ['A_Team_attack', 
               'A_Team_mid',
               'A_Team_def',
               'B_Team_attack', 
               'B_Team_mid', 
               'B_Team_def']
             

team_ratings_df = pd.DataFrame(data, index=feature_cols)

In [28]:
team_ratings_df = team_ratings_df.transpose()
team_ratings_df['HomeGoals'] = league_results_df['HomeGoals']
team_ratings_df['AwayGoals'] = league_results_df['AwayGoals']
team_ratings_df['result'] = league_results_df['Result']

In [29]:
team_ratings_df.dropna(inplace=True)

In [30]:
team_ratings_df.head()

,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def,HomeGoals,AwayGoals,result
0,85.0,83.0,83.0,80.0,78.0,78.0,4,3,HomeWin
1,76.0,75.0,75.0,89.0,83.0,85.0,0,2,AwayWin
2,86.0,85.0,84.0,75.0,77.0,75.0,2,3,AwayWin
3,78.0,76.0,79.0,75.0,73.0,74.0,0,3,AwayWin
4,81.0,81.0,82.0,78.0,79.0,77.0,1,0,HomeWin


In [31]:
home_team_df = team_ratings_df[home_cols].copy()
home_team_df.columns = rename_cols
away_team_df = team_ratings_df[away_cols].copy()
away_team_df.columns = rename_cols

X = pd.concat([home_team_df, away_team_df]).reset_index(drop=True)

In [32]:
X.tail()

,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def
2887,88.0,86.0,88.0,81.0,81.0,80.0
2888,79.0,80.0,79.0,80.0,80.0,81.0
2889,78.0,78.0,78.0,86.0,85.0,83.0
2890,80.0,80.0,80.0,86.0,85.0,86.0
2891,79.0,77.0,77.0,79.0,80.0,80.0


In [33]:
print(X.shape)
X.iloc[[0,1370]]

(2892, 6)


,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def
0,85.0,83.0,83.0,80.0,78.0,78.0
1370,80.0,78.0,78.0,81.0,81.0,81.0


In [159]:
result_mapping_dict = {'HomeWin': 2, 'Draw': 1, 'AwayWin': 0}
y = pd.DataFrame(league_results_df['Result']).reset_index(drop=True).replace(result_mapping_dict)
y.head()

,Result
0,2
1,0
2,0
3,0
4,2


In [160]:
print(y.shape)


(1446, 1)


# Machine Learning

In [158]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

ValueError: Found input variables with inconsistent numbers of samples: [2892, 1446]

In [132]:
# default split is 75% for training and 25% for testing
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2169, 6)
(2169, 1)
(723, 6)
(723, 1)


In [133]:
# import model
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=1)
clf = clf.fit(X_train, y_train)

# instantiate
linreg = LinearRegression()
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)
logreg.fit(X_train, y_train)
clf.fit(X_train, y_train)

/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/Users/michaelstrenk/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [134]:
# # print the intercept and coefficients
# print(linreg.intercept_)
# print(linreg.coef_)

# print(logreg.intercept_)
# print(logreg.coef_)

In [135]:
# pair the feature names with the coefficients
# list(zip(feature_cols, logreg.coef_))

In [155]:
# make predictions on the testing set
linreg_y_pred = linreg.predict(X_test)
logreg_y_pred = logreg.predict(X_test)
clf_y_pred = clf.predict(X_test)

In [156]:
# from sklearn import metrics
# print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [157]:
linreg_y_pred

array([[1.0393278 ],
       [1.2082783 ],
       [1.18336025],
       [1.55617699],
       [1.58568382],
       [1.255572  ],
       [1.51200574],
       [1.38121535],
       [1.36128621],
       [1.3307164 ],
       [1.11253367],
       [0.88359125],
       [2.03820301],
       [1.7074245 ],
       [0.91317135],
       [0.72609167],
       [2.29110022],
       [0.93050174],
       [1.0677259 ],
       [1.19274524],
       [0.84545925],
       [1.09034248],
       [2.39285159],
       [1.09337401],
       [2.06044256],
       [1.05181594],
       [1.18505785],
       [1.38690169],
       [2.06950716],
       [0.80998645],
       [1.04737458],
       [0.73540779],
       [1.66704929],
       [0.57951061],
       [1.30541073],
       [1.28342889],
       [1.19343825],
       [0.99650977],
       [2.13497165],
       [0.9451404 ],
       [1.85056461],
       [1.31772588],
       [2.26730164],
       [1.65211832],
       [0.78833646],
       [1.31510565],
       [1.1994819 ],
       [0.986

In [121]:
# import seaborn

In [122]:
# seaborn.distplot(y_pred)
# plt.show()

In [123]:
# seaborn.distplot(test_model_df['real_results'])
# plt.show()

In [124]:
# seaborn.distplot(test_model_df['normalized predictions'])
# plt.show()

In [147]:
ml_data = [linreg_y_pred, logreg_y_pred, clf_y_pred]

In [150]:
test_model_df = pd.DataFrame(ml_data, 
                             index=['lin Reg', 'log Reg', 'clf']
                            )

In [151]:
# test_model_df['predictions'].value_counts()
test_model_df

,0
lin Reg,"[[1.0393278033188662], [1.2082783035545361], [..."
log Reg,"[0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
clf,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [53]:
# def normalize_test_model(y):
#     if y > (4/3):
#         return 2
#     elif y < (2/3):
#         return 0
#     else:
#         return 1

In [55]:
# counter = 0

# for index, row in test_model_df.iterrows():
#     if row['real_results'] == row['predictions']:
#         counter += 1
#     else:
#         continue
        
        
# counter / len(test_model_df)

In [56]:
# test_model_df['normalized predictions'] = test_model_df['predictions'].apply(normalize_test_model)

In [57]:
# count = 0

# for index, row in test_model_df.iterrows():
#     if row['real_results'] == row['normalized predictions']:
#         count += 1
        
# count / len(test_model_df)

# Poisson

In [167]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

home_team_df = team_ratings_df[home_cols].copy()
home_team_df.columns = rename_cols
away_team_df = team_ratings_df[away_cols].copy()
away_team_df.columns = rename_cols

X = pd.concat([home_team_df, away_team_df]).reset_index(drop=True)

y = pd.DataFrame(pd.concat([team_ratings_df['HomeGoals'], team_ratings_df['AwayGoals']]).reset_index(drop=True), columns=['goals'])

goal_model_data = X.join(y)

goal_model_data.head()

,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def,goals
0,85.0,83.0,83.0,80.0,78.0,78.0,4
1,76.0,75.0,75.0,89.0,83.0,85.0,0
2,86.0,85.0,84.0,75.0,77.0,75.0,2
3,78.0,76.0,79.0,75.0,73.0,74.0,0
4,81.0,81.0,82.0,78.0,79.0,77.0,1


In [169]:
poisson_model = smf.glm(formula="goals ~ A_Team_attack + A_Team_mid + A_Team_def + B_Team_attack + B_Team_mid + B_Team_def", 
                        data=goal_model_data, 
                        family=sm.families.Poisson()).fit()

poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 2892
Model:                            GLM   Df Residuals:                     2885
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -4221.6
Date:                Wed, 13 Jun 2018   Deviance:                       3404.4
Time:                        10:35:48   Pearson chi2:                 3.01e+03
No. Iterations:                     5                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.6821      0.507     -3.317      0.001      -2.676      -0.688
A_Team_attack     0.0434      0.011      3.934      0.000       0.022       0.065
A_Team_mid        0.0020      0.012      0.170      0.865      -0.021       0.025
A_Team_def        0.0259      0.012      2.113      0.035       0.002       0.050
B_Team_attack    -0.0129      0.011     -1.149      0.250      -0.035       0.009
B_Team_mid       -0.0359      0.012     -2.903      0.004      -0.060      -0.012
B_Team_def        0.0020      0.013      0.155      0.877      -0.023       0.027
=================================================================================
"""

In [59]:
# team_data_test1 = {'A_Team_attack': 84,
#                   'A_Team_mid': 84,
#                   'A_Team_def': 85,
#                   'B_Team_attack': 80, 
#                   'B_Team_mid': 78, 
#                   'B_Team_def': 80}


# poisson_model.predict(pd.DataFrame(data=team_data_test1, index=[1]))


1    1.932062
dtype: float64

In [60]:


# team_data_test2 = {'A_Team_attack': 80,
#                   'A_Team_mid': 78,
#                   'A_Team_def': 80,
#                   'B_Team_attack': 84, 
#                   'B_Team_mid': 84, 
#                   'B_Team_def': 85}


# poisson_model.predict(pd.DataFrame(data=team_data_test2, index=[1]))

1    1.090379
dtype: float64

# Running Game Simulations

In [170]:
def simulate_goals(game, a_team='A Team', b_team='B Team', verbose=False):
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals


def show_game_summary(game, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(game, a_team, b_team, verbose=verbose)
    
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals


def simulate_match(foot_model, a_team, b_team, type='Club', max_goals=9, verbose=False):
    a_team_stats = get_team_stats(a_team, type=type)
    b_team_stats = get_team_stats(b_team, type=type)
    
    data1 = {'A_Team_attack': a_team_stats[0],
              'A_Team_mid': a_team_stats[1],
              'A_Team_def': a_team_stats[2],
              'B_Team_attack': b_team_stats[0], 
              'B_Team_mid': b_team_stats[1], 
              'B_Team_def': b_team_stats[2]}
    
    data2 = {'A_Team_attack': b_team_stats[0],
              'A_Team_mid': b_team_stats[1],
              'A_Team_def': b_team_stats[2],
              'B_Team_attack': a_team_stats[0], 
              'B_Team_mid': a_team_stats[1], 
              'B_Team_def': a_team_stats[2]}
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [a_goals_avg, b_goals_avg]]
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, a_team, b_team, verbose=verbose)
    return game


In [171]:
game = simulate_match(poisson_model, 'Belgium', 'Germany', type='Nationality')
show_game_summary(game, 'Belgium', 'Germany', verbose=True)

Belgium chance of Win: 33.21%
Germany chance of Win: 42.5%
Odds of a draw: 24.29%
Random float: 0.22017988971807123
Belgium goals: 0
Germany goals: 3



(0.3320872363735382, 0.42498530193176426, 0.24291869883834877, 0, 3)

In [64]:
simulate_goals(game)

(3, 0)

## Getting the world cup squad Stats

In [65]:
get_team_stats('Belgium', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Belgium     Attack                  86.0
            Defense                 86.0
            Midfield                84.0

In [66]:
get_team_stats('Sweden', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Sweden      Attack                  80.0
            Defense                 78.0
            Midfield                80.0

In [67]:
schedule_df = pd.read_csv('resources/worldcup_match_schedule.csv')
schedule_df.head()

,a_team,b_team
0,Russia,Saudi Arabia
1,Egypt,Uruguay
2,Portugal,Spain
3,Morocco,Iran
4,France,Australia


In [68]:
def get_bulk_team_stats(schedule):


    a_team_attack = []
    a_team_mid = []
    a_team_def = []
    b_team_attack = []
    b_team_mid = []
    b_team_def = []
    data = []


    for index, row in schedule.iterrows():
        try:
            a_team_stats = get_team_stats(row['a_team'], type='Nationality')
            a_attack = a_team_stats[0][0]
            a_midfield = a_team_stats[0][1]
            a_defense = a_team_stats[0][2]

            b_team_stats = get_team_stats(row['b_team'], type='Nationality')
            b_attack = b_team_stats[0][0]
            b_midfield = b_team_stats[0][1]
            b_defense = b_team_stats[0][2]

            if len(a_team_stats[0]) != 3 or len(b_team_stats[0]) != 3:
                a_team_attack.append(None)
                a_team_mid.append(None)
                a_team_def.append(None)
                b_team_attack.append(None)
                b_team_mid.append(None)
                b_team_def.append(None)
            else:            
                a_team_attack.append(a_attack)
                a_team_mid.append(a_midfield)
                a_team_def.append(a_defense)
                b_team_attack.append(b_attack)
                b_team_mid.append(b_midfield)
                b_team_def.append(b_defense)


        except IndexError:
            a_team_attack.append(None)
            a_team_mid.append(None)
            a_team_def.append(None)
            b_team_attack.append(None)
            b_team_mid.append(None)
            b_team_def.append(None)
            continue

    data = [a_team_attack, 
            a_team_mid, 
            a_team_def, 
            b_team_attack,
            b_team_mid, 
            b_team_def]

    rename_cols = ['A_Team_attack', 
                   'A_Team_mid',
                   'A_Team_def',
                   'B_Team_attack', 
                   'B_Team_mid', 
                   'B_Team_def']


    return pd.DataFrame(data, index=rename_cols).transpose()

In [69]:
# mask = overall_score_df['Nationality'] == 'Serbia'
# groupby_cols = ['Nationality', 'Preferred Positions']
# player_overall_df[mask].head(13).groupby(groupby_cols).mean().round(0)

In [70]:
def get_game_summary(df):

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []

    for index, row in schedule_df.iterrows():
        a_team = row[0]
        b_team = row[1]

        game = simulate_match(poisson_model, a_team, b_team, type='Nationality')
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)
        
    
    df['a_team_chances'] = pd.Series(a_team_odds)
    df['b_team_chances'] = pd.Series(b_team_odds)
    df['draw_chances'] = pd.Series(draw_odds)
    df['a_goals'] = pd.Series(a_goals_count)
    df['b_goals'] = pd.Series(b_goals_count)
    
    return df

In [71]:
def calculate_stage_winner(df):
    a_team_points = []
    b_team_points = []

    for index, row in group_stage_results_df.iterrows():
        win = 3
        loss = 0
        draw = 1
        if row['a_goals'] == row['b_goals']:
            a_team_points.append(draw)
            b_team_points.append(draw)
        elif row['a_goals'] > row['b_goals']:
            a_team_points.append(win)
            b_team_points.append(loss)
        elif row['a_goals'] < row['b_goals']:
            a_team_points.append(loss)
            b_team_points.append(win)


        df['a_points'] = pd.Series(a_team_points)
        df['b_points'] = pd.Series(b_team_points)
        
    return df

group_stage_results_df = get_game_summary(schedule_df)
group_stage_results_df = calculate_stage_winner(group_stage_results_df)
group_stage_results_df

,a_team,b_team,a_team_chances,b_team_chances,draw_chances,a_goals,b_goals,a_points,b_points
0,Russia,Saudi Arabia,0.554290,0.202677,0.243023,2,0,3,0
1,Egypt,Uruguay,0.128553,0.675456,0.195929,2,3,0,3
2,Portugal,Spain,0.311893,0.449925,0.238170,1,2,0,3
3,Morocco,Iran,0.632917,0.147130,0.219929,2,1,3,0
4,France,Australia,0.783094,0.075025,0.141530,3,1,3,0
5,Peru,Denmark,0.210984,0.542639,0.246369,1,2,0,3
6,Argentina,Iceland,0.823846,0.058962,0.116335,1,0,3,0
7,Croatia,Nigeria,0.597850,0.175881,0.226247,3,0,3,0
8,Brazil,Switzerland,0.611717,0.175833,0.212408,1,0,3,0
9,Costa Rica,Serbia,0.214069,0.543399,0.242522,1,3,0,3


In [72]:
cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
        'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False]).reset_index(drop=True)
scoreboard_df

,Group,Team,Goals,Points
0,A,Uruguay,6,7
1,A,Saudi Arabia,5,4
2,A,Egypt,4,3
3,A,Russia,3,3
4,B,Spain,11,9
5,B,Portugal,4,4
6,B,Morocco,4,3
7,B,Iran,2,1
8,C,France,5,7
9,C,Denmark,4,5


In [ ]:
round_of_16_games = [(0, 5), (8, 13),
                     (16, 21), (24, 29),
                     (4, 1), (12, 9),
                     (20, 17), (28, 25)]

round_of_8_games = [(0, 1), (2, 3),
                     (4, 5), (6, 7)]

round_of_4_games = [(0, 1), (2, 3)]

round_of_2_games = [(0, 1)]

def get_team_penalties(team):
    mask = players_df['Nationality'] == team
    return pd.to_numeric(players_df[mask]['Penalties'].head(), errors='coerce').mean()

def simulate_penalties(a_team, b_team):
    a_pen = get_team_penalties(a_team)
    b_pen = get_team_penalties(b_team)
    
    if a_pen == b_pen:
        winner = a_team
    elif a_pen > b_pen:
        winner = a_team
    elif a_pen < b_pen:
        winner = b_team
    
    return winner

def get_knockout_winner(round_results_df):
    winners = []
    
    for index, row in round_results_df.iterrows():
        if row['a_goals'] == row['b_goals']:
            winner = simulate_penalties(row[0], row[1])
        elif row['a_goals'] > row['b_goals']:
            winner = row[0]
        elif row['a_goals'] < row['b_goals']:
            winner = row[1]
            
        winners.append(winner)
        
    round_results_df['Winner'] = pd.Series(winners)
    
    return round_results_df
    

def simulate_knockout_stage(df, schedule_tuples):
    a_team_lists = []
    b_team_lists = []

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []
    
    
    for match in schedule_tuples:
        a_team = df.iloc[match[0]]['Team']
        b_team = df.iloc[match[1]]['Team']
        a_team_lists.append(a_team)
        b_team_lists.append(b_team)

        game = simulate_match(poisson_model, a_team, b_team, type="Nationality")
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)

    round_results_df = pd.DataFrame([a_team_lists, b_team_lists]).transpose()
    round_results_df = get_game_summary(round_results_df)
    round_results_df = get_knockout_winner(round_results_df)
    
    df = pd.DataFrame(round_results_df['Winner']).rename(columns={'Winner': 'Team'})    
    
    return df



In [ ]:
def simulate_World_Cup(n_simulations=None, verbose=False):
    world_cup_winners = []
    n = 0
    while n < n_simulations:
        try:
            group_stage_results_df = get_game_summary(schedule_df)
            group_stage_results_df = calculate_stage_winner(group_stage_results_df)

            cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
                    'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

            a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
            b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

            scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
            scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
            scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
            scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False])

            df_8 = simulate_knockout_stage(scoreboard_df, round_of_16_games)
            df_4 = simulate_knockout_stage(df_8, round_of_8_games)
            df_2 = simulate_knockout_stage(df_4, round_of_4_games)
            world_cup_winner = simulate_knockout_stage(df_2, round_of_2_games).iloc[0].item()

            world_cup_winners.append(world_cup_winner)
            
            if verbose == True:
                print(group_stage_results_df)
                print(scoreboard_df)
                print(df_8)
                print(df_4)
                print(df_2)
                print(pd.Series(world_cup_winners).value_counts())
                
            print('The winner of World Cup 2018 simulation {}/{} is {}'.format(n+1, n_simulations, world_cup_winner))
            

            n += 1
        except:
            print('Exception occured')
            continue
        
    return pd.Series(world_cup_winners).value_counts()
        


In [ ]:
# world_cup_results = simulate_World_Cup(1000, verbose=False)

In [ ]:
world_cup_results